In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pickle


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score


In [ ]:
experiments = [('bgl', True, True, 0.001, False),
 ('bgl', True, False, 0.001, False),
 ('bgl', False, True, 0.001, False),
 ('bgl', False, False, 0.001, False),
 ('bgl', True, True, 0.05, False),
 ('bgl', True, False, 0.05, False),
 ('bgl', False, True, 0.05, False),
 ('bgl', False, False, 0.05, False),
 ('bgl', True, True, 0.1, False),
 ('bgl', True, False, 0.1, False),
 ('bgl', False, True, 0.1, False),
 ('bgl', False, False, 0.1, False),
 ('bgl', True, True, 0.2, False),
 ('bgl', True, False, 0.2, False),
 ('bgl', False, True, 0.2, False),
 ('bgl', False, False, 0.2, False),
 ('bgl', True, True, 0.4, False),
 ('bgl', True, False, 0.4, False),
 ('bgl', False, True, 0.4, False),
 ('bgl', False, False, 0.4, False),
 ('bgl', True, True, 0.6, False),
 ('bgl', True, False, 0.6, False),
 ('bgl', False, True, 0.6, False),
 ('bgl', False, False, 0.6, False),
 ('bgl', True, True, 0.8, False),
 ('bgl', True, False, 0.8, False),
 ('bgl', False, True, 0.8, False),
 ('bgl', False, False, 0.8, False)]


In [ ]:
from collections import defaultdict

def evaluate_greater(labels_test, max_distances, start_epoch, end_epoch):

    res_ = defaultdict(dict)
    
    for epoch in range(start_epoch, end_epoch):
        print("Best results per epoch {}".format(epoch+1))
        print("-------"*10)
        f1_scores = []
        precision_scores = []
        recall_scores = []
        auc_score = []
        d = {}
        fpr, tpr, thresholds = roc_curve(labels_test, max_distances[epoch], pos_label=1)
        
#         if len(thresholds)>300:
#             top_th = 100
#         else:
#             top_th = 50
        top_th = 50
        
        
#         top_th = 30
        print("Number of th to evaluate {}".format(top_th))
#         candidate_th = np.sort(np.abs(np.gradient(thresholds)))[(-1)*top_th:]
#         candidate_th_indx = np.where((tpr - (1-fpr))==0, True, False)
#         print(candidate_th_indx)
        print("MEANS ", np.argmax(np.sqrt(tpr * (1-fpr))))
        
        candidate_th = np.array(thresholds[np.argmax(np.sqrt(tpr * (1-fpr)))]).reshape(-1, 1)
        
        candidate_th = np.array(thresholds[np.argsort((tpr-fpr))][(-1)*top_th:]).reshape(-1, 1)
        
#         print(type(candidate_th))
# #         plt.figure()
# #         plt.plot(labels_test, c="b")
# #         print(np.unique(labels_test, return_counts=True))
# #         plt.plot(max_distances[epoch],c="r")
# #         plt.figure()
# #         plt.scatter(fpr, tpr)
# #         plt.show()
        
# #         plt.figure(1)
#         plt.plot(tpr, c="b")
#         plt.plot(1-fpr, c="r")
#         plt.plot((tpr - (1-fpr)))
#         plt.scatter(np.arange(tpr.shape[0]), tpr-fpr, c="g")
#         plt.scatter(np.argsort((tpr-fpr))[(-1)*top_th:], (tpr-fpr)[np.argsort((tpr-fpr))[(-1)*top_th:]], c="purple")
#         plt.show()
#         plt.plot(thresholds)
#         plt.plot(np.abs(np.gradient(thresholds)))
#         plt.show()
        
#         plt.figure(2)
#         plt.scatter(fpr, tpr)
# #         plt.show()
#         plt.scatter(np.arange(0,1, 0.05), np.arange(0,1, 0.05))
#         plt.show()
#         plt.figure(1)
#         plt.scatter(np.arange(len(max_distances[epoch])), max_distances[epoch])
#         plt.show()
#         plt.figure(2)
#         plt.scatter(np.arange(len(max_distances[epoch])), labels_test)
#         plt.show()
#         print(np.unique(candidate_th))
#         print(thresholds)
        best_f1 = 0
        preds = []
        for th in candidate_th:
            preds = np.where((max_distances[epoch]>th)==True, 1, 0)
            print("The F1 score is {} the threshold is {}".format(f1_score(labels_test, preds), th))
#             auc_score.append(roc_auc_score(labels_test, max_distances[epoch]))
            f1_scores.append(f1_score(labels_test, preds))
            if f1_score(labels_test, preds) > best_f1:
                best_preds = preds
                best_f1 = f1_score(labels_test, preds)
#             precision_scores.append(precision_score(labels_test, preds))
#             recall_scores.append(recall_score(labels_test, preds))
    
        d["f1"] = np.max(f1_scores)
        d["precision"] = precision_score(labels_test, best_preds)
        d["recall"] = recall_score(labels_test, best_preds)
        d["thresholds"] = candidate_th[np.argmax(f1_scores)]
        d["AUC"] = roc_auc_score(labels_test, max_distances[epoch])
        ind = np.argmax(f1_scores)
        d["best_index_per_epoch"] = ind
        print("THE AUC score is {}".format(roc_auc_score(labels_test, max_distances[epoch])))
#         print("THE MAXIMAL VALUE FOR the th is {}".format(candidate_th[ind]))
        
#         d["precision"] = precision_scores[ind]
#         d["recall"] = recall_scores[ind]
#         d["auc"] = auc_score[ind]
        res_[epoch] = d
    
    return res_


In [ ]:
experiment = ('spirit', True, True, 0.2, False)

In [ ]:
dataset_name = experiment[0]

path = "./log_anomaly_detection_evaluation/" + dataset_name + "/"
model_name = "transfer_"
phase = "distances_"

with open (path + model_name + phase + "_".join([str(x) for x in experiment]) + ".pickle", "rb") as file:
    max_distances = pickle.load(file)

path = "./log_anomaly_detection_evaluation/" + dataset_name + "/"
model_name = "transfer_"
phase = "labels_test_"

with open (path + model_name + phase + "_".join([str(x) for x in experiment]) + ".pickle", "rb") as file:
    labels_test = pickle.load(file)

In [ ]:
def read_experiment(experiment):
    dataset_name = experiment[0]

    path = "./log_anomaly_detection_evaluation/" + dataset_name + "/run1/"
    model_name = "transfer_"
    phase = "distances_"
    
    print("reading distances")
    
    with open (path + model_name + phase + "_".join([str(x) for x in experiment]) + ".pickle", "rb") as file:
        max_distances = pickle.load(file)

    print("reading labels")
    
    path = "./log_anomaly_detection_evaluation/" + dataset_name + "/run1/"
    model_name = "transfer_"
    phase = "labels_test_"

    with open (path + model_name + phase + "_".join([str(x) for x in experiment]) + ".pickle", "rb") as file:
        labels_test = pickle.load(file)
    
    
    print("calculating_results")
    res_ = evaluate_greater(labels_test, max_distances, start_epoch=0, end_epoch=len(max_distances))
    
    print("------"*10)
    
    return res_
    


In [ ]:
set_exp = defaultdict(dict)

for idx, experiment in enumerate(experiments):
    print("_".join([str(x) for x in experiment]))
    set_exp["_".join([str(x) for x in experiment])] = read_experiment(experiment)